In [1]:
import h5py 
import numpy as np
import matplotlib.pyplot as plt

This code basically takes your visibility data in the shape ($N_{times},N_{freq},N_{correlations}$) and rearranges it so that it is in the required shape ($N_{blts},N_{spws},N_{freq},N_{pol}$). This requires taking the vis data and reorganising it according to dish correlations and polarisations in the order (xx,yy,xy,yx).

In [2]:


#This function is used to find the array index corresponding to a given (a,b) input
#and is used for the prod array in the hirax file (['index_map/prod']).
def orig_array_ind(orig_array,a,b):
    ind = np.argwhere((orig_array['input_a'] == a) & (orig_array['input_b'] == b))[0][0]
    return ind

def index_array(function,orig_array):
    row,col=function.shape
    flat=np.ndarray.flatten(function)
    indices=np.array([])
    for i in flat:
        a,b=i[0],i[1]
        index=orig_array_ind(orig_array,a,b)
        indices=np.append(indices,index)
    indices=np.reshape(indices,(row,col))
    return indices


def Vis_reshape(prod_array,Ndish):
    Nbls_full=np.int((Ndish*(Ndish-1)/2)+Ndish) #with autos
    Nbls_cross=np.int((Ndish*(Ndish-1)/2)) #without autos
    Nfeeds=Ndish*2
    array_even=np.array([],[('input_a', '<u2'), ('input_b', '<u2')])
    for i in prod_array:
        if i[0]%2==0:
            array_even_single=i
            array_even=np.append(array_even,array_even_single)
    array_even=np.reshape(array_even,(Nbls_full,2))
    array_odd_auto=np.array([],[('input_a', '<u2'), ('input_b', '<u2')])
    for m in prod_array:
        if m[0]%2==1:
            if m[0]==m[1]:
                array_odd_auto_single=m
                array_odd_auto=np.append(array_odd_auto,array_odd_auto_single)
                array_odd_auto=array_odd_auto[:,np.newaxis]
    array_even_auto=np.array([],[('input_a', '<u2'), ('input_b', '<u2')])
    for x in array_even:
        if x[0][0]==x[0][1]:
            array_even_auto_single=x
            array_even_auto=np.append(array_even_auto,array_even_auto_single)
    array_even_auto=np.reshape(array_even_auto,(Ndish,2))
    array_even_auto=np.hstack((array_even_auto,array_odd_auto))
    permutation=[0,2,1]
    array_even_auto=array_even_auto[:,permutation]
    array_even_auto_indices=index_array(array_even_auto,prod_array)
    #array_even=np.append(array_even,)
    array_even_cross=np.array([],[('input_a', '<u2'), ('input_b', '<u2')])
    for x in array_even:
        if x[0][0]!=x[0][1]:
            array_even_cross_single=x
            array_even_cross=np.append(array_even_cross,array_even_cross_single)
    array_even_cross=np.reshape(array_even_cross,(Nbls_cross,2))
    #array_even_cross=np.hstack((array_even_cross,array_odd_auto))
    #print (array_new)
    array_odd=np.array([],[('input_a', '<u2'), ('input_b', '<u2')])
    for j in prod_array:
        if j[0]%2==1:
            if j[0]!=j[1]:
                array_odd_single=j
                array_odd=np.append(array_odd,array_odd_single)
    array_odd=np.reshape(array_odd,(Nbls_cross,2))
    array_even_cross=np.hstack((array_even_cross,array_odd))
    permutation = [0,3,1,2]
    array_even_cross=array_even_cross[:,permutation]
    array_even_cross_indices=index_array(array_even_cross,prod_array)
    auto_flatten=np.ndarray.flatten(array_even_auto_indices)
    cross_flatten=np.ndarray.flatten(array_even_cross_indices)
    vis_permutation_array=np.concatenate((auto_flatten,cross_flatten))
    vis_permutation_array=vis_permutation_array.astype(int)
    vis_permutation_list=np.ndarray.tolist(vis_permutation_array)
    return vis_permutation_list

def vis_auto_conj(vis_data_rearranged,index):
    col = vis_data_rearranged[:,:,index]

    #cojugate column
    #to conjugate x value set ind=0, to conjugate y value set ind=1
    ind = 1
    col_flat = col.flatten()
    col_conj = np.copy(col_flat)

    for i in range(len(col_flat)):
        col_conj[i][ind] = -1*col_flat[i][ind]	
        #col_conj[i] = np.conj(col_flat[i][ind])

    col_conj = col_conj.reshape(col.shape)

    #insert column in A (np.insert is surprisingly uselful)
    col_pos = index+1
    vis_data_rearranged = np.insert(vis_data_rearranged,col_pos,col_conj,axis=2)
    return vis_data_rearranged

In [3]:
file_hirax=h5py.File('00197942_0000.h5','r')

In [4]:
list(file_hirax.keys())

['flags', 'fpga_hk', 'gain_coeff', 'gain_exp', 'index_map', 'vis']

In [5]:
file_hirax['index_map']['freq']

<HDF5 dataset "freq": shape (1024,), type "|V16">

In [6]:
prod=file_hirax['index_map']['prod']
vis=file_hirax['vis']

In [31]:
permutation=Vis_reshape(prod,8)

In [33]:
vis_perm=[]
for i in permutation:
    vis_perm.append(np.transpose(vis[:,:,i]))
new_vis_data=np.array(vis_perm).T
#np.save('vis_permutated',new_vis_data)

In [36]:
new_vis_data=np.load('vis_permutated.npy')
Ndish=8
new_vis_data_0=vis_auto_conj(new_vis_data,2)
new_vis_data_1=vis_auto_conj(new_vis_data_0,6)
new_vis_data_2=vis_auto_conj(new_vis_data_1,10)
new_vis_data_3=vis_auto_conj(new_vis_data_2,14)
new_vis_data_4=vis_auto_conj(new_vis_data_3,18)
new_vis_data_5=vis_auto_conj(new_vis_data_4,22)
new_vis_data_6=vis_auto_conj(new_vis_data_5,26)
new_vis_data_7=vis_auto_conj(new_vis_data_6,30)
print (new_vis_data_7)
#np.save('hirax_vis_full',new_vis_data_7)


[[[( 205499841,          0) ( 205520896,          0)
   (-205494765,          0) ... ( 205520896,          0)
   ( 205520896,          0) ( 205520896,          0)]
  [( 180702779,          0) ( 192602956,          0)
   (  22001410,    6313121) ... (  31500001,    3925715)
   (   8669628,   -2637173) (   8346638,   -4439090)]
  [( 181353988,          0) ( 147019567,          0)
   (   -728743,   -1754895) ... (  39587966,   -8770685)
   (  -8077423,    4579756) (   3359686,    1425725)]
  ...
  [( 383242321,          0) ( 379744023,          0)
   ( 353331305,    4685238) ... (-125180318,  -85989107)
   ( -34563028,   85937092) (  14525125,  -35211290)]
  [( 263239166,          0) ( 155939434,          0)
   (  41458808,  -22056999) ... ( -34779409,   -5762383)
   (   5913309,    1858770) (   2856294,   -2134633)]
  [( 259882133,          0) ( 169546446,          0)
   (  36099208,  -13486379) ... ( -14206807,    4380581)
   (   -746723,   -1516934) (   5230595,    3095044)]]

 [[( 205

In [22]:
vis_full=np.load('hirax_vis_full.npy')
vis_final=np.reshape(vis_full,(1024*36,1,1024,4))
#np.save('vis_final',vis_final)

In [26]:
vis_final=np.load('vis_final.npy')
print (vis_final.shape)

(36864, 1, 1024, 4)


In [7]:
mat=np.array([(101,1),(102,2),(103,3),(104,4),(105,5),(106,6),(107,7),(108,8),(109,9),(110,10),(111,11),(112,12)],[('input_a', '<i4'), ('input_b', '<i4')])
mat=np.reshape(mat,(2,2,3))
print (mat.shape)
print (mat)
#np.save('temp_mat',mat)
#np.load('temp_mat.npy')

(2, 2, 3)
[[[(101,  1) (102,  2) (103,  3)]
  [(104,  4) (105,  5) (106,  6)]]

 [[(107,  7) (108,  8) (109,  9)]
  [(110, 10) (111, 11) (112, 12)]]]


In [5]:
permutation=[0,2,1]
print (type(permutation))

<class 'list'>


In [6]:
mat[:,:,permutation]

array([[[(101,  1), (103,  3), (102,  2)],
        [(104,  4), (106,  6), (105,  5)]],

       [[(107,  7), (109,  9), (108,  8)],
        [(110, 10), (112, 12), (111, 11)]]],
      dtype=[('input_a', '<i4'), ('input_b', '<i4')])

In [7]:
mat_perm=[]
for i in permutation:
    mat_perm.append(np.transpose(mat[:,:,i]))
print (np.array(mat_perm).T)
    #for j in range(0,3):
     #   print (j)
      #  print (mat_perm[:,:,j])
       # mat_perm[:,:,j]=mat[:,:,i]

[[[(101,  1) (103,  3) (102,  2)]
  [(104,  4) (106,  6) (105,  5)]]

 [[(107,  7) (109,  9) (108,  8)]
  [(110, 10) (112, 12) (111, 11)]]]


In [19]:
j=0
mat_new=vis_auto_conj(mat,j)
mat_new_2=vis_auto_conj(mat_new,j+2)
mat_new_3=vis_auto_conj(mat_new_2,j+4)
print (mat_new_3)
mat_new_3=np.reshape(mat_new_3,(2*2,1,2,3))
#plt.imshow(mat_new_3[:,:,1]['input_a'])
print (mat_new_3[:,:,0])

[[[(101,   1) (101,  -1) (102,   2) (102,  -2) (103,   3) (103,  -3)]
  [(104,   4) (104,  -4) (105,   5) (105,  -5) (106,   6) (106,  -6)]]

 [[(107,   7) (107,  -7) (108,   8) (108,  -8) (109,   9) (109,  -9)]
  [(110,  10) (110, -10) (111,  11) (111, -11) (112,  12) (112, -12)]]]
[[[(101,   1) (101,  -1) (102,   2)]]

 [[(104,   4) (104,  -4) (105,   5)]]

 [[(107,   7) (107,  -7) (108,   8)]]

 [[(110,  10) (110, -10) (111,  11)]]]


In [158]:
v1=np.array([(0,13),(0,14),(0,15),(0,16)],dtype=[('input_a', 'float'), ('input_b', 'float')])

In [73]:
v1=np.reshape(v1,(2,2))

[[[(101,   1) (102,   2) (102,  -2) (103,   3)]
  [(104,   4) (105,   5) (105,  -5) (106,   6)]]

 [[(107,   7) (108,   8) (108,  -8) (109,   9)]
  [(110,  10) (111,  11) (111, -11) (112,  12)]]]


In [28]:
file_hirax.close()

In [51]:
file_tmp=h5py.File('temp_file','w')

In [54]:
g1=file_tmp.create_group('group1')

In [60]:
list(file_tmp.keys())

['group1']

In [59]:
del file_tmp['gr1']

In [61]:
del file_tmp['group1']

In [62]:
g1=file_tmp.create_group('group1/real')

In [63]:
g2=file_tmp.create_group('group1/imag')

In [76]:
g1.create_dataset('data_1',data=a)

<HDF5 dataset "data_1": shape (2, 2, 3), type "<u2">

In [77]:
g2.create_dataset('data_2',data=b)

<HDF5 dataset "data_2": shape (2, 2, 3), type "<u2">

In [82]:
list(file_tmp['group1']['imag'])

['data_2']

In [83]:
del file_tmp['group1/imag/data_2']

In [84]:
del file_tmp['group1/imag']

In [86]:
del file_tmp['group1/real/data_1']

In [97]:
del file_tmp['group1/real']

KeyError: "Couldn't delete link (callback link pointer is NULL (specified link may be '.' or not exist))"

In [96]:
g1.create_dataset('real',data=a)

RuntimeError: Unable to create link (name already exists)

In [89]:
g1.create_dataset('imag',data=a)

<HDF5 dataset ("anonymous"): shape (2, 2, 3), type "<u2">

In [98]:
list(file_tmp['group1'])

[]